# Training ResNeXt

This Notebook will go through the process of training the ResNeXt model with an architecture that is optimized for the Cifar Dataset (as specified in [1] https://arxiv.org/pdf/1611.05431.pdf).

In [1]:
import os
import sys
import json
import shutil
import torch
import torch.nn.functional as F
import torchvision.datasets as dset
import torchvision.transforms as transforms
import time

In [2]:
sys.path.append("..") # Note, this line is needed only since this notebook is being run from inside 'bharat' directory
                      # If you run this notebook in the same folder as the the 'models-py-code' directory, remove this line
from pymodels.resnext import CifarResNeXt # This is where we actually refer to the model we wish to train

## Set all required parameters
Remember to change the `session_id` for each new model you wish to train or if you change of the other parameters

In [3]:
args={}

# Input Data Parameters
args['data_path'] = '/datasets/ee285s-public/'
args['dataset'] = 'cifar100' # The other option is 'cifar10'

# Optimization options
args['epochs'] = 300
args['start_epoch'] = 0 # Change only if necessary. Used for resuming sessions
args['batch_size'] = 96
args['learning_rate'] = 0.1
args['momentum'] = 0.9 # Momentum
args['decay'] = 0.0005 # Weight decay (L2 penalty)
args['test_bs'] = 64 # Test Batch Size
args['schedule'] = [75, 150] # Decrease learning rate at these epochs
args['gamma'] = 0.1 # LR is multiplied by gamma on schedule.

# Checkpoints and Session Parameters
args['save_dir'] = './' # Folder to save checkpoint file and best model file
args['load'] = 'resnext-0-checkpoint.pth' # Path to load Checkpoint file 
args['session_id'] = '0' # Remember to change the session id for each new model you train

# Architecture
args['depth'] = 29 # Model depth
args['cardinality'] = 8 # Model cardinality (group)
args['base_width'] = 64 # Number of channels in each group
args['widen_factor'] = 4 # Widen factor. 4 -> 64, 8 -> 128, ...

# Acceleration
args['ngpu'] = 1 # 0 = CPU
args['prefetch'] = 2 # Pre-fetching threads

# i/o
args['log_dir'] = './log/' # Log folder

### Initialize Logging

In [4]:
if not os.path.isdir(args['log_dir']):
        os.makedirs(args['log_dir'])
if args['load'] == '':
    log = open(os.path.join(args['log_dir'], 'resnext-{}-log.txt'.format(args['session_id'])), 'w')
else:
    log = open(os.path.join(args['log_dir'], 'resnext-{}-log.txt'.format(args['session_id'])), 'a')

### Setup the Input Data

In [5]:
mean = [x / 255 for x in [125.3, 123.0, 113.9]]
std = [x / 255 for x in [63.0, 62.1, 66.7]]

train_transform = transforms.Compose(
    [transforms.RandomHorizontalFlip(),
     transforms.RandomCrop(32, padding=4),
     transforms.ToTensor(),
     transforms.Normalize(mean, std)])

test_transform = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize(mean, std)])

if args['dataset'] == 'cifar10':
    train_data = dset.CIFAR10(args['data_path'], train=True, transform=train_transform, download=False)
    test_data = dset.CIFAR10(args['data_path'], train=False, transform=test_transform, download=False)
    nlabels = 10
else:
    train_data = dset.CIFAR100(args['data_path'], train=True, transform=train_transform, download=False)
    test_data = dset.CIFAR100(args['data_path'], train=False, transform=test_transform, download=False)
    nlabels = 100

train_loader = torch.utils.data.DataLoader(train_data, batch_size=args['batch_size'], shuffle=True,
                                                                       num_workers=args['prefetch'], pin_memory=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=args['test_bs'], shuffle=False,
                                                                     num_workers=args['prefetch'], pin_memory=True)

### Setup the Model, Criterion, and Optimizer

In [6]:
resnext_model = CifarResNeXt(args['cardinality'], args['depth'], nlabels, args['base_width'], args['widen_factor'])
print(resnext_model)
if args['ngpu'] > 1:
    resnext_model = torch.nn.DataParallel(resnext_model, device_ids=list(range(args['ngpu'])))

if args['ngpu'] > 0:
    resnext_model.cuda()

optimizer = torch.optim.SGD(resnext_model.parameters(), args['learning_rate'], momentum=args['momentum'],
                            weight_decay=args['decay'], nesterov=True)

CifarResNeXt(
  (conv_1_3x3): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn_1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
  (stage_1): Sequential(
    (stage_1_bottleneck_0): ResNeXtBottleneck(
      (conv_reduce): Conv2d(64, 512, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn_reduce): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True)
      (conv_conv): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=8, bias=False)
      (bn): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True)
      (conv_expand): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn_expand): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
      (shortcut): Sequential(
        (shortcut_conv): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (shortcut_bn): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True)
      )
    )
    (stage_1_bottleneck_1): ResNeXtBottleneck(
      (

### Setup the Train and Test Functions

In [7]:
def train(epoch):
    resnext_model.train()
    loss_avg = 0.0
    t0 = time.time()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = torch.autograd.Variable(data.cuda()), torch.autograd.Variable(target.cuda())

        # forward
        output = resnext_model(data)

        # backward
        optimizer.zero_grad()
        t0 = time.time()
        loss = F.cross_entropy(output, target)
        t1 = time.time()
        loss.backward()
        optimizer.step()

        # exponential moving average
        loss_avg = loss_avg * 0.2 + float(loss) * 0.8

        print("===> Epoch[{}]({}/{}): Loss: {:.4f} || Timer: {:.4f} sec.".format(epoch, batch_idx, len(train_loader), loss.data[0], (t1 - t0)))

    epoch_state['train_loss'] = loss_avg
    t1 = time.time()
    print("===> Epoch {} Complete: Avg. Loss: {:.4f} | Time: {:.4f}".format(epoch, loss_avg, (t1-t0)))

# test function (forward only)
def test():
    resnext_model.eval()
    loss_avg = 0.0
    correct = 0
    for batch_idx, (data, target) in enumerate(test_loader):
        data, target = torch.autograd.Variable(data.cuda()), torch.autograd.Variable(target.cuda())

        # forward
        output = resnext_model(data)
        loss = F.cross_entropy(output, target)

        # accuracy
        pred = output.data.max(1)[1]
        correct += float(pred.eq(target.data).sum())

        # test loss average
        loss_avg += float(loss)

    epoch_state['test_loss'] = loss_avg / len(test_loader)
    epoch_state['test_accuracy'] = correct / len(test_loader.dataset)

### Define a function to save a checkpoint

In [8]:
def save_checkpoint(state, is_best, filename='resnext-{}-checkpoint.pth'.format(args['session_id'])):
    filepath = os.path.join(args['save_dir'], filename)
    torch.save(state, filepath)
    if is_best:
        bestfilepath = os.path.join(args['save_dir'], 'model-best-resnext-{}.pth'.format(args['session_id']))
        shutil.copyfile(filepath, bestfilepath)

### Load and Resume from checkpoint (if set)

In [9]:
if not args['load'] == '':
    if os.path.isfile(args['load']):
        print("=> Loading Checkpoint '{}'".format(args['load']))
        checkpoint = torch.load(args['load'])
        args['start_epoch'] = checkpoint['epoch'] + 1
        resnext_model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        print("=> Loaded checkpoint '{}' (epoch {})"
              .format(args['load'], checkpoint['epoch']))
    else:
        print("=> No checkpoint found at '{}'".format(args['load']))

=> Loading Checkpoint 'resnext-0-checkpoint.pth'
=> Loaded checkpoint 'resnext-0-checkpoint.pth' (epoch 38)


### Main Loop

In [ ]:
best_accuracy = 0.0
epoch_state = {'args' : args}
for epoch in range(args['start_epoch'], args['epochs']):
    if epoch in args['schedule']:
        epoch_state['learning_rate'] *= args['gamma']
        for param_group in optimizer.param_groups:
            param_group['lr'] = epoch_state['learning_rate']

    epoch_state['epoch'] = epoch
    train(epoch)
    test()
    if epoch_state['test_accuracy'] > best_accuracy:
        best_accuracy = epoch_state['test_accuracy']
        isbest = True
    
    save_checkpoint({
        'epoch' : epoch,
        'state_dict' : resnext_model.state_dict(),
        'optimizer' : optimizer.state_dict()
    }, isbest)
    
    log.write('%s\n' % json.dumps(epoch_state))
    log.flush()
    #print(epoch_state)
    print("[Epoch {}] Best accuracy: {:.4f}".format(epoch, best_accuracy))

log.close()

===> Epoch[39](0/521): Loss: 0.9516 || Timer: 0.0015 sec.
===> Epoch[39](1/521): Loss: 1.2045 || Timer: 0.0004 sec.
===> Epoch[39](2/521): Loss: 0.7910 || Timer: 0.0002 sec.
===> Epoch[39](3/521): Loss: 1.0417 || Timer: 0.0003 sec.
===> Epoch[39](4/521): Loss: 1.0221 || Timer: 0.0005 sec.
===> Epoch[39](5/521): Loss: 0.9995 || Timer: 0.0003 sec.
===> Epoch[39](6/521): Loss: 0.9543 || Timer: 0.0002 sec.
===> Epoch[39](7/521): Loss: 1.0648 || Timer: 0.0005 sec.
===> Epoch[39](8/521): Loss: 1.1851 || Timer: 0.0003 sec.
===> Epoch[39](9/521): Loss: 0.9705 || Timer: 0.0002 sec.
===> Epoch[39](10/521): Loss: 1.2722 || Timer: 0.0003 sec.
===> Epoch[39](11/521): Loss: 0.8576 || Timer: 0.0004 sec.
===> Epoch[39](12/521): Loss: 0.8488 || Timer: 0.0002 sec.
===> Epoch[39](13/521): Loss: 0.8266 || Timer: 0.0002 sec.
===> Epoch[39](14/521): Loss: 1.1461 || Timer: 0.0002 sec.
===> Epoch[39](15/521): Loss: 0.8802 || Timer: 0.0003 sec.
===> Epoch[39](16/521): Loss: 0.8214 || Timer: 0.0003 sec.
===> Ep

===> Epoch[39](139/521): Loss: 0.9178 || Timer: 0.0002 sec.
===> Epoch[39](140/521): Loss: 0.9447 || Timer: 0.0002 sec.
===> Epoch[39](141/521): Loss: 0.8855 || Timer: 0.0002 sec.
===> Epoch[39](142/521): Loss: 0.9994 || Timer: 0.0002 sec.
===> Epoch[39](143/521): Loss: 1.3385 || Timer: 0.0002 sec.
===> Epoch[39](144/521): Loss: 1.1263 || Timer: 0.0002 sec.
===> Epoch[39](145/521): Loss: 1.0224 || Timer: 0.0002 sec.
===> Epoch[39](146/521): Loss: 0.8236 || Timer: 0.0002 sec.
===> Epoch[39](147/521): Loss: 1.1117 || Timer: 0.0002 sec.
===> Epoch[39](148/521): Loss: 1.0006 || Timer: 0.0002 sec.
===> Epoch[39](149/521): Loss: 1.0253 || Timer: 0.0002 sec.
===> Epoch[39](150/521): Loss: 0.8203 || Timer: 0.0002 sec.
===> Epoch[39](151/521): Loss: 0.9378 || Timer: 0.0001 sec.
===> Epoch[39](152/521): Loss: 1.3077 || Timer: 0.0002 sec.
===> Epoch[39](153/521): Loss: 1.2198 || Timer: 0.0002 sec.
===> Epoch[39](154/521): Loss: 0.9018 || Timer: 0.0002 sec.
===> Epoch[39](155/521): Loss: 1.1103 ||

===> Epoch[39](492/521): Loss: 1.3189 || Timer: 0.0002 sec.
===> Epoch[39](493/521): Loss: 1.2800 || Timer: 0.0002 sec.
===> Epoch[39](494/521): Loss: 1.3123 || Timer: 0.0002 sec.
===> Epoch[39](495/521): Loss: 0.9740 || Timer: 0.0002 sec.
===> Epoch[39](496/521): Loss: 0.9982 || Timer: 0.0002 sec.
===> Epoch[39](497/521): Loss: 1.1815 || Timer: 0.0002 sec.
===> Epoch[39](498/521): Loss: 0.7197 || Timer: 0.0002 sec.
===> Epoch[39](499/521): Loss: 1.1837 || Timer: 0.0002 sec.
===> Epoch[39](500/521): Loss: 1.1266 || Timer: 0.0002 sec.
===> Epoch[39](501/521): Loss: 1.0225 || Timer: 0.0001 sec.
===> Epoch[39](502/521): Loss: 1.2788 || Timer: 0.0001 sec.
===> Epoch[39](503/521): Loss: 1.0202 || Timer: 0.0001 sec.
===> Epoch[39](504/521): Loss: 1.2430 || Timer: 0.0001 sec.
===> Epoch[39](505/521): Loss: 0.9925 || Timer: 0.0003 sec.
===> Epoch[39](506/521): Loss: 1.1391 || Timer: 0.0002 sec.
===> Epoch[39](507/521): Loss: 0.8131 || Timer: 0.0002 sec.
===> Epoch[39](508/521): Loss: 1.2451 ||

===> Epoch[40](108/521): Loss: 0.9781 || Timer: 0.0001 sec.
===> Epoch[40](109/521): Loss: 1.1028 || Timer: 0.0002 sec.
===> Epoch[40](110/521): Loss: 0.8473 || Timer: 0.0002 sec.
===> Epoch[40](111/521): Loss: 1.0487 || Timer: 0.0002 sec.
===> Epoch[40](112/521): Loss: 1.1214 || Timer: 0.0002 sec.
===> Epoch[40](113/521): Loss: 1.2631 || Timer: 0.0002 sec.
===> Epoch[40](114/521): Loss: 0.6777 || Timer: 0.0002 sec.
===> Epoch[40](115/521): Loss: 0.9806 || Timer: 0.0002 sec.
===> Epoch[40](116/521): Loss: 0.8694 || Timer: 0.0002 sec.
===> Epoch[40](117/521): Loss: 1.2120 || Timer: 0.0002 sec.
===> Epoch[40](118/521): Loss: 1.1012 || Timer: 0.0002 sec.
===> Epoch[40](119/521): Loss: 0.8917 || Timer: 0.0002 sec.
===> Epoch[40](120/521): Loss: 0.9228 || Timer: 0.0002 sec.
===> Epoch[40](121/521): Loss: 0.7710 || Timer: 0.0002 sec.
===> Epoch[40](122/521): Loss: 0.8642 || Timer: 0.0002 sec.
===> Epoch[40](123/521): Loss: 1.0127 || Timer: 0.0002 sec.
===> Epoch[40](124/521): Loss: 0.8187 ||

===> Epoch[40](245/521): Loss: 1.0421 || Timer: 0.0002 sec.
===> Epoch[40](246/521): Loss: 0.9235 || Timer: 0.0002 sec.
===> Epoch[40](247/521): Loss: 0.8060 || Timer: 0.0002 sec.
===> Epoch[40](248/521): Loss: 0.6631 || Timer: 0.0002 sec.
===> Epoch[40](249/521): Loss: 1.0758 || Timer: 0.0002 sec.
===> Epoch[40](250/521): Loss: 0.7525 || Timer: 0.0002 sec.
===> Epoch[40](251/521): Loss: 1.2306 || Timer: 0.0002 sec.
===> Epoch[40](252/521): Loss: 0.9444 || Timer: 0.0002 sec.
===> Epoch[40](253/521): Loss: 1.0682 || Timer: 0.0002 sec.
===> Epoch[40](254/521): Loss: 1.1285 || Timer: 0.0002 sec.
===> Epoch[40](255/521): Loss: 1.1052 || Timer: 0.0002 sec.
===> Epoch[40](256/521): Loss: 1.0515 || Timer: 0.0002 sec.
===> Epoch[40](257/521): Loss: 1.0172 || Timer: 0.0002 sec.
===> Epoch[40](258/521): Loss: 1.1004 || Timer: 0.0002 sec.
===> Epoch[40](259/521): Loss: 1.1600 || Timer: 0.0002 sec.
===> Epoch[40](260/521): Loss: 1.0207 || Timer: 0.0002 sec.
===> Epoch[40](261/521): Loss: 1.0508 ||

...

===> Epoch[48](51/521): Loss: 0.8150 || Timer: 0.0002 sec.
===> Epoch[48](52/521): Loss: 0.6374 || Timer: 0.0002 sec.
===> Epoch[48](53/521): Loss: 1.2087 || Timer: 0.0002 sec.
===> Epoch[48](54/521): Loss: 0.7995 || Timer: 0.0002 sec.
===> Epoch[48](55/521): Loss: 0.8563 || Timer: 0.0002 sec.
===> Epoch[48](56/521): Loss: 0.8746 || Timer: 0.0003 sec.
===> Epoch[48](57/521): Loss: 0.8467 || Timer: 0.0003 sec.
===> Epoch[48](58/521): Loss: 0.9528 || Timer: 0.0002 sec.
===> Epoch[48](59/521): Loss: 0.9391 || Timer: 0.0002 sec.
===> Epoch[48](60/521): Loss: 0.8218 || Timer: 0.0002 sec.
===> Epoch[48](61/521): Loss: 1.0422 || Timer: 0.0002 sec.
===> Epoch[48](62/521): Loss: 1.0042 || Timer: 0.0002 sec.
===> Epoch[48](63/521): Loss: 0.8910 || Timer: 0.0002 sec.
===> Epoch[48](64/521): Loss: 0.9166 || Timer: 0.0002 sec.
===> Epoch[48](65/521): Loss: 0.8731 || Timer: 0.0002 sec.
===> Epoch[48](66/521): Loss: 0.9012 || Timer: 0.0002 sec.
===> Epoch[48](67/521): Loss: 0.9260 || Timer: 0.0002 se

===> Epoch[48](189/521): Loss: 1.2605 || Timer: 0.0002 sec.
===> Epoch[48](190/521): Loss: 1.0774 || Timer: 0.0002 sec.
===> Epoch[48](191/521): Loss: 1.0967 || Timer: 0.0002 sec.
===> Epoch[48](192/521): Loss: 1.0177 || Timer: 0.0002 sec.
===> Epoch[48](193/521): Loss: 0.9702 || Timer: 0.0003 sec.
===> Epoch[48](194/521): Loss: 1.3191 || Timer: 0.0002 sec.
===> Epoch[48](195/521): Loss: 0.9044 || Timer: 0.0002 sec.
===> Epoch[48](196/521): Loss: 0.9597 || Timer: 0.0002 sec.
===> Epoch[48](197/521): Loss: 0.8548 || Timer: 0.0002 sec.
===> Epoch[48](198/521): Loss: 1.2703 || Timer: 0.0002 sec.
===> Epoch[48](199/521): Loss: 1.1106 || Timer: 0.0002 sec.
===> Epoch[48](200/521): Loss: 1.0933 || Timer: 0.0002 sec.
===> Epoch[48](201/521): Loss: 1.2437 || Timer: 0.0002 sec.
===> Epoch[48](202/521): Loss: 0.8442 || Timer: 0.0002 sec.
===> Epoch[48](203/521): Loss: 0.8449 || Timer: 0.0002 sec.
===> Epoch[48](204/521): Loss: 0.9806 || Timer: 0.0002 sec.
===> Epoch[48](205/521): Loss: 1.0650 ||

===> Epoch[48](326/521): Loss: 1.0409 || Timer: 0.0002 sec.
===> Epoch[48](327/521): Loss: 0.8718 || Timer: 0.0002 sec.
===> Epoch[48](328/521): Loss: 0.8239 || Timer: 0.0002 sec.
===> Epoch[48](329/521): Loss: 1.0652 || Timer: 0.0002 sec.
===> Epoch[48](330/521): Loss: 1.0753 || Timer: 0.0002 sec.
===> Epoch[48](331/521): Loss: 0.9749 || Timer: 0.0002 sec.
===> Epoch[48](332/521): Loss: 0.9642 || Timer: 0.0002 sec.
===> Epoch[48](333/521): Loss: 1.0244 || Timer: 0.0002 sec.
===> Epoch[48](334/521): Loss: 1.1924 || Timer: 0.0002 sec.
===> Epoch[48](335/521): Loss: 1.0041 || Timer: 0.0002 sec.
===> Epoch[48](336/521): Loss: 0.9755 || Timer: 0.0002 sec.
===> Epoch[48](337/521): Loss: 0.7882 || Timer: 0.0002 sec.
===> Epoch[48](338/521): Loss: 1.4328 || Timer: 0.0002 sec.
===> Epoch[48](339/521): Loss: 1.1511 || Timer: 0.0002 sec.
===> Epoch[48](340/521): Loss: 1.0737 || Timer: 0.0002 sec.
===> Epoch[48](341/521): Loss: 1.2708 || Timer: 0.0002 sec.
===> Epoch[48](342/521): Loss: 1.0061 ||

===> Epoch[48](463/521): Loss: 0.9321 || Timer: 0.0002 sec.
===> Epoch[48](464/521): Loss: 0.9219 || Timer: 0.0002 sec.
===> Epoch[48](465/521): Loss: 0.9374 || Timer: 0.0002 sec.
===> Epoch[48](466/521): Loss: 1.1414 || Timer: 0.0002 sec.
===> Epoch[48](467/521): Loss: 0.8819 || Timer: 0.0002 sec.
===> Epoch[48](468/521): Loss: 1.2697 || Timer: 0.0002 sec.
===> Epoch[48](469/521): Loss: 1.1055 || Timer: 0.0002 sec.
===> Epoch[48](470/521): Loss: 1.1801 || Timer: 0.0002 sec.
===> Epoch[48](471/521): Loss: 1.0187 || Timer: 0.0002 sec.
===> Epoch[48](472/521): Loss: 0.9553 || Timer: 0.0002 sec.
===> Epoch[48](473/521): Loss: 0.9356 || Timer: 0.0002 sec.
===> Epoch[48](474/521): Loss: 0.9938 || Timer: 0.0002 sec.
===> Epoch[48](475/521): Loss: 1.0080 || Timer: 0.0002 sec.
===> Epoch[48](476/521): Loss: 1.1397 || Timer: 0.0002 sec.
===> Epoch[48](477/521): Loss: 0.9860 || Timer: 0.0002 sec.
===> Epoch[48](478/521): Loss: 1.0612 || Timer: 0.0002 sec.
===> Epoch[48](479/521): Loss: 1.0933 ||

===> Epoch[49](79/521): Loss: 0.8061 || Timer: 0.0002 sec.
===> Epoch[49](80/521): Loss: 0.9411 || Timer: 0.0002 sec.
===> Epoch[49](81/521): Loss: 0.8341 || Timer: 0.0002 sec.
===> Epoch[49](82/521): Loss: 0.8954 || Timer: 0.0002 sec.
===> Epoch[49](83/521): Loss: 0.8980 || Timer: 0.0002 sec.
===> Epoch[49](84/521): Loss: 0.9922 || Timer: 0.0002 sec.
===> Epoch[49](85/521): Loss: 0.8444 || Timer: 0.0002 sec.
===> Epoch[49](86/521): Loss: 1.0417 || Timer: 0.0002 sec.
===> Epoch[49](87/521): Loss: 0.9368 || Timer: 0.0002 sec.
===> Epoch[49](88/521): Loss: 0.7884 || Timer: 0.0002 sec.
===> Epoch[49](89/521): Loss: 0.7482 || Timer: 0.0002 sec.
===> Epoch[49](90/521): Loss: 1.0167 || Timer: 0.0002 sec.
===> Epoch[49](91/521): Loss: 1.0351 || Timer: 0.0002 sec.
===> Epoch[49](92/521): Loss: 0.8826 || Timer: 0.0001 sec.
===> Epoch[49](93/521): Loss: 0.8911 || Timer: 0.0001 sec.
===> Epoch[49](94/521): Loss: 0.9961 || Timer: 0.0002 sec.
===> Epoch[49](95/521): Loss: 1.1448 || Timer: 0.0001 se